# 1. Fudamentals
## 1.1. Terms
**Mutex** stands for "mutual exclusion semaphore", which indicates they are good for ensuring **mutual exclusion** between threads when accessing shared resources. Mutexes are similiar to **binary semaphores**.

Recall that all threads running in a process share the resources of that process: memory, files, etc. 
Consider the case when two threads need a shared resource (like a serial port) to perform some action (e.g. send messages in sequences of bytes). We don't want to get the bytes of the messages mixed up coming from the two threads, so this **resource** needs to be under **"mutual exclusion"**: 
* **Only one thread** can access it while sending a complete message. 
* We call the section of the code accessing the shared resources a **"critical section"**. Two or more threads cannot excecute in the critical simultaneously
* **At most** one thread is permitted to execute that critical section at any time.

## 1.2. Mutex States
A mutex has two states: locked an unlocked. It can be considered as a binary counter, that can be either 
* **0 (locked)**: when entering the critical section
* **1(unlocked)**: when exiting the critical section
One can think of two operations to protect the critical section

## 1.3. Semantics
* ```lock(m)```: 
    * If mutex ```m``` is unclocked (counter = 1), lock it (decrease the counter to 0). 
    * If the mutex is locked (counter = 0), block until ```m``` is unlocked.
* ```unlock(m)```: 
    * If mutex ```m``` is unlocked (counter = 1), error
    * If mutex is locked (counter = 0), unlock it (increase the counter) and wake up blocked threads
    
The mutex must be:
* **locked** to **aquire a shared resource** (entering the critical section) before **accessing** it
* **unlocked** when the access to the shared resource is **terminated**

## 1.4. Semaphore vs. Mutex
While a semaphore provides generic synchornization because of the semaphpre can be initialized to a generic value, a mutex explictly provides the concept of **critical section** (they can be **ONLY** used for mutal exclusion). 

The mutex counter is always automatically initialized to 1, and the mutex can be unlocked only by the thread that locked it!

Mutexes are less powerful than (general purpose) semaphores, but can help preventing program errors.

# 2. Basic Mutex Operations
In POSIX, basic mutex operations we covered in this lab are

## 2.1. Initialize/Detrory a mutex
### 2.1.a Define and initialize a mutex
* A mutex is defined by a decriptor of type: ```pthread_mutex_t```
* A mutex must be initialized before using with ```pthread_mutex_init()``` function
    * When initializing a mutex, a **strucutre** of type ```pthread_mutexattr_t``` can be used to describe the mutex attributes 

To initialize a mutex, use:

```int pthread_mutex_init(pthread_mutex_t *mutex, const pthread_mutexattr_t *attr);```

* Returns 0 when success, non-0 in case of error
* The mutex is returned in mutex *(Think about **why**, its a good mini practice if your C skill is rusted)*
* attr can be  ```NULL``` when using the **default attributes**

### 2.1.b. Coding example for mutex initialization
E.g. the code snipet below creates 

* a mutex variable ```mymutex```
* a mutex attributes structure ```mymutexattr```
* intialize mutex attributes with ```pthread_mutexattr_init()```
* initalize mutex with ```pthread_mutex_init()``` 

```c
    pthread_mutex_t mymutex;
    pthread_mutexattr_init(&mymutexattr);
    pthread_mutex_init(&mymutex, &mymutexattr);
```


**Note:** Alternatively, you can initialize the mutex variable with an initializer at the place where the variable is declared like: ```pthread_mutex_t mutex_variable = PTHREAD_MUTEX_INITIALIZER;```. This is equivalent to calling the ```pthread_mutex_init()``` with a ```NULL``` for the ```attr```.

### 2.1.c. Destroy a mutex
If a mutex is no longer needed, use ```int pthread_mutex_destroy(pthread_mutex_t *mutex);```
E.g. destroy the ```mymutex``` we created using: 
```c
pthread_mutex_destroy(&mymutex);
```

## 2.2. Lock/unlock the mutex
### 2.1.a. Blocking locking
We have discussed about it in section 1. It works like:
* ```lock(m)``` with ```int pthread_mutex_lock(pthread_mutex_t *mutex)```
    * If mutex ```m``` is unclocked (counter = 1), lock it (decrease the counter to 0). 
    * If the mutex is locked (counter = 0), block until ```m``` is unlocked.
* ```unlock(m)``` with ```int pthread_mutex_unlock(pthread_mutex_t *mutex)```
    * If mutex ```m``` is unlocked (counter = 1), error
    * If mutex is locked (counter = 0), unlock it (increase the counter) and wake up blocked threads
    
E.g. lock/unlock ```mymutex``` we created before
```c
  pthread_mutex_lock(&mymutex);
  pthread_mutex_unlock(&mymutex);
```

### 2.2.b. Non-blocking locking
Non-blocking locking (called ```trylock```) using ```int pthread_mutex_trylock(pthread_mutex_t *mutex)```. THis locking function works as follows:
* If the mutex is unlocked (=1), lock it (decrease the counter to 0) and continue
* If the mutex is locked (=0), fail without blocking but returning an error
E.g. trylock ```mymutex``` we created
```c
  pthread_mutex_trylock(&mymutex);
```

Note that ```pthread_mutex_lock()``` is not a cancellation point, this is the basic introduction to mutexes.

# 3. Working with multiple mutexes
In some situations you may need to grab multiple resources - e.g. if a thread needs exclusive access to more than one (say 2) devices at the same time.

Device access can be encapsulated into critical sections. but in this case, a thread will need to enter **another critical section** while already **being in a critical section**


The problem for two devices can be solved by using two mutexes, each protecting one of the resources. However, it is important that the two mutexes are **locked** in **the same order** in **each thread**. 

e.g. ```lock(a) -> lock(b) -> unlock(b) -> unlock(a)```

Otherwise, a deadlock may occur. Why?

# 4. Mutexes with timeouts
## 4.1. Why bother ?
A thread can possibly remain blocked for a **loooooong** time on a lock operation if a thread holding the mutex is doing a lengthy calculation. The solution is using the ```timed lock``` operation:

```c
int pthread_mutex_timedlock(pthread_mutex_t *restrict mutex, const struct timespec *restrict abstime);
```

It works similiarly to ```pthread_mutex_lock()```, except that:
* when calling thread blocks the system starts a **timer** which will expire when the system's clock reaches the value specified by the ```abstime``` parameter
* If the mutex is still **locked**, the call returns with an error code.
* If the mutex becomes unlocked earlier than the timeout, the lock operation succeeds; the thread obtains the lock and proceeds.

## 4.2. Usage
The ```struct timespec``` data type is defined in ```<time.h>``` and documented in POSIX documentation. You can read the CPU's current time value using: 

```c
int clock_gettime(clockid_t, struct timespec *now);
```

where the first parameter must be ```CLOCK_REALTIME```. The result ```now``` contains the time value.

The value has to be increased to a larger time value, which can then be passed to the timedlock operation as an argument. 

When that time value arrives and the lock still cannot be locked, the timedlock operation will return an ETIMEOUT error code (defined in ```<errno.h>```)

# Appendix/Playground: Starter Code - ```ex_mutex.c```

Include necessary libraries and headers

In [1]:
#include <unistd.h>
#include <stdio.h>
#include <pthread.h>

static pthread_mutex_t mymutex;
static int th_cnt;

Thread function ```body``` using blocking locking (oppose to trylock)

Plz ignore the warning message. This notebook is using a C++11 kernel

In [2]:
static void *body(void *arg)
{
  int j;
  
  fprintf(stderr, "Thread %d: %s\n", th_cnt++, (char *)arg); 
  pthread_mutex_lock(&mymutex);
  for (j=0; j<20; j++) {
    usleep(5);
    fprintf(stderr,(char *)arg);
  }
  pthread_mutex_unlock(&mymutex);

  return NULL;
}

input_line_9:9:20: warning: format string is not a string literal (potentially insecure) [-Wformat-security]
    fprintf(stderr,(char *)arg);
                   ^       ~~~
input_line_9:9:20: note: treat the string as an argument to avoid this
    fprintf(stderr,(char *)arg);
                   ^
                   "%s", 


```main``` program

In [3]:
pthread_t t1, t2, t3;
int err;

pthread_mutexattr_t mymutexattr;

pthread_mutexattr_init(&mymutexattr);
pthread_mutex_init(&mymutex, &mymutexattr);
pthread_mutexattr_destroy(&mymutexattr);

err = pthread_create(&t1, NULL, body, (void *)".");
err = pthread_create(&t2, NULL, body, (void *)"#");
err = pthread_create(&t3, NULL, body, (void *)"o");

pthread_join(t1, NULL);
pthread_join(t2, NULL);
pthread_join(t3, NULL);

pthread_mutex_destroy(&mymutex);
printf("\n");

Thread 0: .
Thread 1: #
Thread 2: o
....................####################oooooooooooooooooooo